# Pydantic to OpenAI function

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variables
openai_key = os.getenv('API_KEY')
os.environ["OPENAI_API_KEY"]= openai_key
import warnings
warnings.filterwarnings('ignore')

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [5]:
#Normal python class 
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [7]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [9]:
foo.name

'Joe'

In [11]:
# not a valid data to age that's what pydantic classes are used
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [13]:
foo.age

'bar'

In [15]:
#pydantic class
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [17]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [19]:
foo_p.name

'Jane'

## Pydantic to OpenAI function definition

In [22]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [24]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [ ]:
weather_function

## Using it in a chain

In [29]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [ ]:
model = ChatOpenAI()

In [ ]:
chain = prompt | model

In [ ]:
chain.invoke({"input": "what is the weather in sf?"})

## Using multiple functions

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [ ]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [ ]:
model_with_functions = model.bind(functions=functions)

In [ ]:
model_with_functions.invoke("what is the weather in sf?")

In [ ]:
model_with_functions.invoke("what are three songs by taylor swift?")